In [1]:
import json
from io import StringIO
from tab_dataset import Cdataset
import ntv_pandas as npd
import pandas as pd

In [2]:
file_name = "payload_pdc.json"
with open(file_name, "r") as f:
    data = f.read()
payload = pd.read_json(StringIO(json.loads(data)), orient="record").reset_index()
#payload

In [3]:
payload1 = pd.read_json("static_14h12m44s.json", orient="record")
payload2 = pd.read_json("static_14h12m46s.json", orient="record")
payload = pd.concat([payload1, payload2], ignore_index=True).reset_index()
payload = pd.read_json("bulk_evses_payload.json", orient="record").reset_index()

In [4]:
# create a Dataframe with an "index" column (example with json data)
payload = pd.read_json("request_2.json", orient="record").reset_index()

In [5]:
fields_payload = set(payload.columns)
#fields_payload
len(payload)

38

In [6]:
# create the schema with the Qualicharge constraints between fields
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "nom_amenageur"],        "link" : "derived" },
         {"fields": ["id_station_itinerance", "siren_amenageur"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "contact_amenageur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_operateur"],        "link" : "derived" },
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "telephone_operateur"],  "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         {"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "num_pdl"],              "link" : "derived" },
         {"fields": ["id_station_itinerance", "id_station_local"],     "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         {"fields": ["id_station_itinerance", "raccordement"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "date_mise_en_service"], "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" },
         {"fields": ["coordonneesXY", "code_insee_commune"],           "link" : "derived" }
    ]
}
fields_schema = set([field for relation in schema['relationships'] for field in relation['fields']])

In [7]:
# identify the difference between payload and schema
fields_errors = fields_schema - fields_payload
fields_errors

{'raccordement'}

In [8]:
schema_reduce = {'relationships': [relation for relation in schema['relationships'] if not(set(relation['fields']) & fields_errors)]}
#schema_reduce

In [9]:
# calculate the errors dictionnary (keys : schema constraint, values : list of index with a constraint error
dic_errors = Cdataset(payload).check_relationship(schema_reduce)
{key: val for key, val in dic_errors.items() if val}

{'nbre_pdc - id_station_itinerance': (0, 1, 2, 3, 4, 5, 6, 7, 8)}

In [10]:
# example of error
errors = payload.loc[dic_errors['nbre_pdc - id_station_itinerance'], ['nbre_pdc', 'id_station_itinerance', 'id_pdc_itinerance']]
#errors['id_station_itinerance'].unique()

In [11]:
#errors.loc[errors['id_station_itinerance'] == 'FRAU1P00810000', :]
errors

,nbre_pdc,id_station_itinerance,id_pdc_itinerance
0,1,FRAU1P00810000,FRAU1E00810110
1,4,FRAU1P00810000,FRAU1E00810210
2,4,FRAU1P00810000,FRAU1E00810220
3,4,FRAU1P00810000,FRAU1E00810230
4,4,FRAU1P00810000,FRAU1E00810240
5,4,FRAU1P00810000,FRAU1E00810310
6,4,FRAU1P00810000,FRAU1E00810320
7,4,FRAU1P00810000,FRAU1E00810330
8,4,FRAU1P00810000,FRAU1E00810340


In [6]:
from datetime import date, datetime, timedelta
from_date =datetime(2025, 11, 1, 10)
to_date = datetime(2025, 11, 2, 1)
days = [
        from_date + timedelta(days=d) for d in range((to_date - from_date).days + 1)
    ]
days, [from_date + timedelta(days=d) for d in range((to_date - from_date).days + 1)]

[datetime.datetime(2025, 11, 1, 10, 0)]